In [ ]:
from pavlov import *
monitor(-1)

In [ ]:
from pavlov import stats
stats.plot(['*perky-boxes*', '*great-fits', -1], 'elo-mohex', 'μ', skip=4, fill=True, head=360)

In [ ]:
stats.review('*perky-boxes*')

In [ ]:
import pandas as pd
pd.concat({
    'policy': stats.pandas(-1, 'loss.policy', rule='5s'),
    'value': stats.pandas(-1, 'loss.value', rule='5s')}, 1).pipe(lambda df: (df - df.mean())).iloc[250:300].plot(marker='o')

In [ ]:
stats.pandas(-1, 'loss.value', rule='5s').iloc[250:300].plot(marker='o')